In [1]:
import pickle
from pathlib import Path
import numpy as np
import sys

ws_dir = Path('..')

if sys.platform.lower()=='linux':
    datadir = Path('/scratch/users/ianpg/SWIlarge/data')
    workdir = Path('/scratch/users/ianpg/SWIlarge/work')
    MPSdir = datadir.joinpath('lith/sgems/MPS')
    lithdir = datadir.joinpath('lith/sgems/')
    GISdir = datadir.joinpath('GIS')
    priordir = datadir.joinpath('PriorModel')
    modeldir = datadir.joinpath('NM_model')
elif sys.platform.lower()=='darwin':
    lithdir = ws_dir.joinpath('data','lith','sgems')


def load_obj(dirname, name):
    import pickle
    with open(Path(dirname).joinpath(name + '.pkl').as_posix(), 'rb') as f:
        return pickle.load(f)


def save_obj(dirname, obj, name):
    import pickle
    with open(Path(dirname).joinpath(name + '.pkl').as_posix(), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


In [ ]:
outputdir_sisim = lithdir.joinpath('sisim')
if not outputdir_sisim.exists():
    outputdir_sisim.mkdir(parents=True)
outputdir_snesim = lithdir.joinpath('snesim')
if not outputdir_snesim.exists():
    outputdir_snesim.mkdir(parents=True)
    
varlist = load_obj(ws_dir.joinpath('data/PriorModel'), 'varlist')
n_it = len(varlist[varlist.keys()[0]])
# TI_prop_values = np.array([45 ,55 ,65 ,75])
# TI_props = TI_prop_values[varlist['TI_prop']].astype(int)
# varlist['TI_props']=TI_props

In [202]:
def write_run_save_snesim(name,varlist,it):  
    cmd_run = ('RunGeostatAlgorithm  snesim_std::/GeostatParamUtils/XML::<parameters>  '
           '<algorithm name="snesim_std" />     '
           '<Use_Affinity  value="0"  />     <Use_Rotation  value="0"  />     '
           '<Cmin  value="1" />     <Constraint_Marginal_ADVANCED  value="0" />     '
           '<resimulation_criterion  value="-1" />     <resimulation_iteration_nb  value="1" />     <Nb_Multigrids_ADVANCED  value="5" />     '
           '<Debug_Level  value="0" />     <Subgrid_choice  value="0"  />     '
           '<expand_isotropic  value="1"  />     <expand_anisotropic  value="0"  />     '
           '<aniso_factor  value="        " />     <Hard_Data  grid="lith_bin_rot" region="" property="Lith_binary"  />     '
           '<use_pre_simulated_gridded_data  value="0"  />     <Use_ProbField  value="0"  />     '
           '<ProbField_properties count="0"   value=""  />     '
           '<TauModelObject  value="1 1" />     <use_vertical_proportion  value="0"  />     '
           '<GridSelector_Sim value="swi_large_grid_2" region=""  />     '
           '<Property_Name_Sim  value="{name}_{it}" />     <Nb_Realizations  value="1" />     '
           '<Seed  value="{seed}" />     '
           '<PropertySelector_Training  grid="swi_grid_small2" region="" property="{ti_name}"  />     '
           '<Nb_Facies  value="2" />     '
           '<Marginal_Cdf  value="0.33 0.67" />     <Max_Cond  value="30" />     '
           '<Search_Ellipsoid  value="4000 2000 20  90 0 0" />  </parameters>   '.format(
               it=it,
               name=name,
               seed=varlist['seed'][it] + 1, #ones-based
               ti_name='ti_' + str(varlist['TI_props'][it])
           ))
    cmd_save = 'SaveGeostatGrid  swi_large_grid_2::C:/Users/ianpg/Documents/git/SWIlarge/data/lith/sgems/snesim/{name}_{it}.gslib::gslib::0::{name}_{it}__real0'.format(name=name,it=it)
    return cmd_run,cmd_save


def write_run_save_sisim(name,varlist,it):  
    cmd_run = ('RunGeostatAlgorithm  sisim::/GeostatParamUtils/XML::<parameters>  <algorithm name="sisim" />     '
           '<Hard_Data_Grid value="lith_bin_rot" region=""  />     '
           '<Hard_Data_Property  value="Lith_binary"  />     <Assign_Hard_Data  value="1"  />     '
           '<coded_grid value="" region=""  />     <coded_props count="0"   value=""  />     '
           '<Max_Conditioning_Data  value="30" />     <Search_Ellipsoid  value="1000 500 25  90 0 0" />    '
           '<AdvancedSearch  use_advanced_search="0"></AdvancedSearch>    '
           '<Variogram_Median_Ik  nugget="0" structures_count="1"  >    <structure_1  contribution="1"  type="Spherical"   >      '
           '<ranges max="{r_x}"  medium="{r_y}"  min="{r_z}"   />      '
           '<angles x="90"  y="0"  z="0"   />    </structure_1>  </Variogram_Median_Ik>    <Grid_Name value="swi_large_grid_2" region=""  />     '
           '<Property_Name  value="{name}_{it}" />     '
           '<Nb_Realizations  value="1" />     <Seed  value="{seed}" />     <Categorical_Variable_Flag  value="1"  />     '
           '<Nb_Indicators  value="2" />     '
           '<Marginal_Probabilities  value="{prop_comp} {prop}" />     <lowerTailCdf  function ="Power"  extreme ="0"  omega ="3" />    '
           '<upperTailCdf  function ="Power"  extreme ="0"  omega ="0.333" />    '
           '<Median_Ik_Flag  value="1"  />     <Full_Ik_Flag  value="0"  />   </parameters>   '.format(
           r_x=varlist['r_x'][it],
           r_y=varlist['r_y'][it],
           r_z=varlist['r_z'][it],
           sill=varlist['sill'][it],
           seed=varlist['seed'][it],
           prop=round(varlist['prop_400'][it],2), 
           prop_comp= round(1-varlist['prop_400'][it],2), 
           name=name,
           it=it
           ))

    cmd_save = 'SaveGeostatGrid  swi_large_grid_2::C:/Users/ianpg/Documents/git/SWIlarge/data/lith/sgems/sisim/{name}_{it}.gslib::gslib::0::{name}_{it}__real0'.format(name=name,it=it)
    return cmd_run,cmd_save

### SISIM

In [208]:
its = np.arange(0,500)
name = 'sisim400'
fname_commands = outputdir_sisim.joinpath('{name}_{it1}_{it2}_commands.txt'.format(name=name,it1=its[0],it2=its[-1])).as_posix()

with open(fname_commands, 'w') as f:
    f.write('LoadProject  C:/Users/ianpg/Dropbox/temp_convenience/SWIlarge/data/lith/sgems/MPS/mps_180.prj \n')
    for it in its:
        if it%10==0:
            f.write('DeleteObjects  lith_bin_rot::swi_grid_large::swi_grid_small2::swi_large_grid_2 \n')
            f.write('LoadProject  C:/Users/ianpg/Dropbox/temp_convenience/SWIlarge/data/lith/sgems/MPS/mps_180.prj \n')
        cmd_run,cmd_save = write_run_save_sisim(name,varlist,it)
        f.write(cmd_run + ' \n')
        f.write(cmd_save + ' \n')
# call(['sgems-x64', fname_commands], shell=False)

### SNESIM

In [209]:
its = np.arange(0,500)
name = 'mps180'
fname_commands = outputdir_snesim.joinpath('{name}_{it1}_{it2}_commands.txt'.format(name=name,it1=its[0],it2=its[-1])).as_posix()


runyn=False

with open(fname_commands, 'w') as f:
    f.write('LoadProject  C:/Users/ianpg/Dropbox/temp_convenience/SWIlarge/data/lith/sgems/MPS/mps_180.prj \n')
    for it in its:
        if it%10==0:
            f.write('DeleteObjects  lith_bin_rot::swi_grid_large::swi_grid_small2::swi_large_grid_2 \n')
            f.write('LoadProject  C:/Users/ianpg/Dropbox/temp_convenience/SWIlarge/data/lith/sgems/MPS/mps_180.prj \n')
        cmd_run,cmd_save = write_run_save_snesim(name,varlist,it)
        f.write(cmd_run + ' \n')
        f.write(cmd_save + ' \n')
if runyn:
    call(['sgems-x64', fname_commands], shell=False)